In [1]:
!kaggle datasets download hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images


Dataset URL: https://www.kaggle.com/datasets/hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images
License(s): CC0-1.0
 90% 89.0M/98.7M [00:00<00:00, 110MB/s] 
100% 98.7M/98.7M [00:00<00:00, 110MB/s]


In [2]:
import zipfile

with zipfile.ZipFile("melanoma-skin-cancer-dataset-of-10000-images.zip", "r") as zip_ref:
    zip_ref.extractall("melanoma_dataset")


In [3]:
import tensorflow as tf

dataset_dir = "melanoma_dataset"

# Load the dataset
dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    image_size=(224, 224),  # Resize images to 224x224
    batch_size=32          # Process images in batches of 32
)

# Normalize pixel values to [0, 1]
dataset = dataset.map(lambda x, y: (x / 255.0, y))


Found 10605 files belonging to 1 classes.


In [4]:
def split_dataset(dataset, train_ratio=0.7, val_ratio=0.15):
    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    train_size = int(train_ratio * dataset_size)
    val_size = int(val_ratio * dataset_size)

    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size).take(val_size)
    test_ds = dataset.skip(train_size + val_size)

    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = split_dataset(dataset)


In [5]:
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomBrightness

data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal_and_vertical"),  # Randomly flip images
    RandomRotation(0.2),                   # Rotate images up to 20%
    RandomZoom(0.2),                       # Zoom in/out by 20%
    RandomBrightness(0.2)                  # Adjust brightness
])

# Apply augmentation to training data
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))


In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [8]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


In [9]:
# Normalize and resize if needed
train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
val_ds = val_ds.map(lambda x, y: (x / 255.0, y))
test_ds = test_ds.map(lambda x, y: (x / 255.0, y))


In [10]:
base_model = ResNet50(
    weights='imagenet',   # Load pre-trained weights
    include_top=False,    # Exclude the top fully connected layers
    input_shape=(224, 224, 3)  # Define input size
)

# Freeze the base model to retain pre-trained weights
base_model.trainable = False


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Reduces spatial dimensions
    Dense(128, activation='relu'),  # Dense layer for feature learning
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])


In [12]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [13]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [14]:
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,850,113 (90.98 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [15]:
import tensorflow as tf
print("GPUs available:", tf.config.list_physical_devices('GPU'))


GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [16]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [17]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam



In [18]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    verbose=1
)


Epoch 1/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 152s 177ms/step - accuracy: 1.0000 - loss: 0.0131 - val_accuracy: 1.0000 - val_loss: 2.0815e-05
Epoch 2/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 83s 104ms/step - accuracy: 1.0000 - loss: 1.9043e-05 - val_accuracy: 1.0000 - val_loss: 1.4311e-05
Epoch 3/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 24s 105ms/step - accuracy: 1.0000 - loss: 1.3066e-05 - val_accuracy: 1.0000 - val_loss: 9.9032e-06
Epoch 4/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 40s 99ms/step - accuracy: 1.0000 - loss: 9.1015e-06 - val_accuracy: 1.0000 - val_loss: 7.0681e-06
Epoch 5/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 42s 103ms/step - accuracy: 1.0000 - loss: 6.5463e-06 - val_accuracy: 1.0000 - val_loss: 5.2094e-06
Epoch 6/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 41s 104ms/step - accuracy: 1.0000 - loss: 4.8583e-06 - val_accuracy: 1.0000 - val_loss: 3.9486e-06
Epoch 7/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 40s 102ms/step - accuracy: 1.0000 - loss: 3.7039e-06 - val_accuracy: 1.0000 - val_loss: 3.0636e-06
Epoch 8/10
232/232 ━━━━━━━━━━━━

In [19]:
# Unfreeze the top layers of the base model
base_model.trainable = True

# Optionally freeze the first few layers to retain low-level features
for layer in base_model.layers[:100]:  # Adjust layer numbers based on model depth
    layer.trainable = False

# Recompile the model with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train again with fine-tuning
fine_tune_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,  # Fine-tuning epochs
    verbose=1
)


Epoch 1/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 76s 189ms/step - accuracy: 1.0000 - loss: 1.8301e-06 - val_accuracy: 1.0000 - val_loss: 2.5420e-06
Epoch 2/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 65s 175ms/step - accuracy: 1.0000 - loss: 3.1824e-08 - val_accuracy: 1.0000 - val_loss: 1.3883e-07
Epoch 3/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 40s 172ms/step - accuracy: 1.0000 - loss: 1.6324e-08 - val_accuracy: 1.0000 - val_loss: 1.2944e-08
Epoch 4/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 41s 172ms/step - accuracy: 1.0000 - loss: 1.0595e-08 - val_accuracy: 1.0000 - val_loss: 8.1346e-09
Epoch 5/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 41s 172ms/step - accuracy: 1.0000 - loss: 7.6506e-09 - val_accuracy: 1.0000 - val_loss: 6.1946e-09
Epoch 6/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 41s 172ms/step - accuracy: 1.0000 - loss: 5.9006e-09 - val_accuracy: 1.0000 - val_loss: 4.9675e-09
Epoch 7/10
232/232 ━━━━━━━━━━━━━━━━━━━━ 41s 172ms/step - accuracy: 1.0000 - loss: 4.7670e-09 - val_accuracy: 1.0000 - val_loss: 4.1064e-09
Epoch 8/10
232/232 ━━━━━━━━